<a href="https://colab.research.google.com/github/donw385/DS-Unit-4-Sprint-2-NLP/blob/master/module3-Document-Classification/LS_DS_423_Document_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Now it's your turn!

Use the following dataset of scraped "Data Scientist" and "Data Analyst" job listings to create your own Document Classification Models.

<https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/job_listings.csv>

Requirements:

- Apply both CountVectorizer and TfidfVectorizer methods to this data and compare results
- Use at least two different classification models to compare differences in model accuracy
- Try to "Hyperparameter Tune" your model by using different n_gram ranges, max_results, and data cleaning methods
- Try and get the highest accuracy possible!

In [2]:
!pip install unidecode

     |████████████████████████████████| 245kB 2.8MB/s 


In [10]:
import codecs

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

from scipy.cluster.hierarchy import dendrogram, linkage
from unidecode import unidecode
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
jobs = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/job_listings.csv')
jobs = jobs.drop_duplicates(subset='description')

In [6]:
jobs = jobs.dropna(axis=0)
jobs.isnull().sum().sum()

0

In [0]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return re.sub(r'^b\"', '', cleantext)

def stem_text(text):
    sentences = sent_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemm_text = []
    for sentence in sentences:
        token_words=word_tokenize(sentence)
        lemm_sentence=[]
        for word in token_words:
            lemm_sentence.append(lemmatizer.lemmatize(unidecode(word)))
            lemm_sentence.append(" ")
        lemm_text.append(''.join(lemm_sentence))
    return unidecode(''.join(lemm_text))

jobs['description'] = jobs['description'].apply(cleanhtml).apply(stem_text)

In [0]:
X = jobs.description
y = jobs.job.map({'Data Scientist' : 1, 'Data Analyst' : 0})

stop = stopwords.words('english')
stop.extend(['x83', 'x98', 'xef', 'x83', 'xc2', 'xa8', 'xe2', 'x80', 'nc3',
            'x99s', 'xa2'])

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.2,
                                                   stratify=y)

In [13]:

lr_count = make_pipeline(CountVectorizer(stop_words=stop),
                        LogisticRegression(solver='lbfgs',
                                          max_iter=500))
lr_grid_params = [{'countvectorizer__ngram_range' : [(1,1), (1,2), (1,3)],
                  'countvectorizer__max_features' : [50, 100, None]}]

lr_grid = GridSearchCV(lr_count, lr_grid_params, cv=3)
lr_grid.fit(X_train, y_train)
print ('Best Params', lr_grid.best_params_)
print ('CV Score', lr_grid.best_score_)
print ('Test Score', lr_grid.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Params {'countvectorizer__max_features': None, 'countvectorizer__ngram_range': (1, 2)}
CV Score 0.9201183431952663
Test Score 0.8470588235294118


In [14]:
nb_count = make_pipeline(CountVectorizer(stop_words=stop),
                        MultinomialNB())
nb_grid_params = [{'countvectorizer__ngram_range' : [(1,1), (1,2), (1,3)],
                  'countvectorizer__max_features' : [50, 100, None]}]

nb_grid = GridSearchCV(nb_count, nb_grid_params, cv=3)
nb_grid.fit(X_train, y_train)
print ('Best Params', nb_grid.best_params_)
print ('CV Score', nb_grid.best_score_)
print ('Test Score', nb_grid.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Params {'countvectorizer__max_features': 100, 'countvectorizer__ngram_range': (1, 2)}
CV Score 0.9201183431952663
Test Score 0.8705882352941177


In [15]:
lr_tfidf = make_pipeline(TfidfVectorizer(stop_words=stop),
                        LogisticRegression(solver='lbfgs',
                                          max_iter=500))
lr_grid_params = [{'tfidfvectorizer__ngram_range' : [(1,1), (1,2), (1,3)],
                  'tfidfvectorizer__max_features' : [50, 100, None]}]
lr_grid = GridSearchCV(lr_tfidf, lr_grid_params, cv=3)
lr_grid.fit(X_train, y_train)
print ('Best Params', lr_grid.best_params_)
print ('CV Score', lr_grid.best_score_)
print ('Test Score', lr_grid.score(X_test, y_test))

Best Params {'tfidfvectorizer__max_features': 100, 'tfidfvectorizer__ngram_range': (1, 2)}
CV Score 0.9142011834319527
Test Score 0.8588235294117647


In [16]:
nb_tfidf = make_pipeline(TfidfVectorizer(stop_words=stop),
                        MultinomialNB())
nb_grid_params = [{'tfidfvectorizer__ngram_range' : [(1,1), (1,2), (1,3)],
                  'tfidfvectorizer__max_features' : [50, 100, None]}]
nb_grid = GridSearchCV(nb_tfidf, nb_grid_params, cv=3)
nb_grid.fit(X_train, y_train)
print ('Best Params', nb_grid.best_params_)
print ('CV Score', nb_grid.best_score_)
print ('Test Score', nb_grid.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Params {'tfidfvectorizer__max_features': 100, 'tfidfvectorizer__ngram_range': (1, 2)}
CV Score 0.9112426035502958
Test Score 0.8705882352941177


In [18]:
nb_pipe = make_pipeline(TfidfVectorizer(stop_words=stop,
                                       max_features=100,
                                       ngram_range=(1,2)),
                       LogisticRegression(solver='lbfgs',
                                          max_iter=500))
nb_pipe.fit(X_train, y_train)
print ('Train Score', nb_pipe.score(X_train, y_train))
print ('Test Score', nb_pipe.score(X_test, y_test))

Train Score 0.9378698224852071
Test Score 0.8588235294117647


In [19]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

svc_pipe = make_pipeline(TfidfVectorizer(stop_words=stop,
                                       max_features=100,
                                       ngram_range=(1,2)),
                       LinearSVC())
# svc_pipe.fit(X_train, y_train)
# print ('Train Score', svc_pipe.score(X_train, y_train))
# print ('Test Score', svc_pipe.score(X_test, y_test))

cross_val_score(svc_pipe, X, y, scoring='roc_auc', cv=5)

array([0.95016611, 0.96843854, 0.96899225, 0.98014748, 0.95632445])

# Stretch Goals

- Try some agglomerative clustering using cosine-similarity-distance! (works better with high dimensional spaces) robust clustering - Agglomerative clustering like Ward would be cool. Try and create an awesome Dendrogram of the most important terms from the dataset.

- Awesome resource for clustering stretch goals: 
 - Agglomerative Clustering with Scipy: <https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/>
 - Agglomerative Clustering for NLP: <http://brandonrose.org/clustering>
 
- Use Latent Dirichlet Allocation (LDA) to perform topic modeling on the dataset: 
 - Topic Modeling and LDA in Python: <https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24>
 - Topic Modeling and LDA using Gensim: <https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/>
